This is an ipython notebook, I run this code in Google Colab, and used google drive to load and save files. You may need to change the way the program load and save files according to your specific scenario, especially the file paths.

In [13]:
!pip install deprecated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from PIL import Image
import pylab
import cv2
import numpy
import scipy
from scipy.ndimage import filters
import scipy.misc as misc
import imageio
import math
import os
from deprecated import deprecated

In [15]:
from google.colab import drive

drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [16]:
path = [[r'/gdrive/MyDrive/stereo_vision/corridorl.jpg',r'/gdrive/MyDrive/stereo_vision/corridorr.jpg'],
    [r'/gdrive/MyDrive/stereo_vision/triclopsi2l.jpg',r'/gdrive/MyDrive/stereo_vision/triclopsi2r.jpg']]

In [21]:
def gen_repeat_matrix(M,margin_size):
    '''
    [ M[h-ms:,w-ms:],M[h-ms:,:],M[h-ms:,:ms]
     M[:,w-ms:],  M[:,:],  M[:,:ms]
     M[:ms,w-ms:], M[:ms,:], M[:ms,:ms] ]
    '''
    h,w = M.shape
    ms = margin_size
    x1 = numpy.concatenate([M[h-ms:,w-ms:],M[h-ms:,:],M[h-ms:,:ms]],axis = 1)
    x2 = numpy.concatenate([M[:,w-ms:],  M[:,:],  M[:,:ms]],axis=1)
    x3 = numpy.concatenate([M[:ms,w-ms:], M[:ms,:], M[:ms,:ms]],axis=1)
    x = numpy.concatenate([x1,x2,x3],axis=0)
    return x

def gen_padded_matrix(M,margin_size,vertical_window=False):
    if vertical_window:
        return numpy.pad(M,((2*margin_size,margin_size),(2*margin_size,margin_size)),mode="constant",constant_values=0)
    else:
        return numpy.pad(M,margin_size,mode="constant",constant_values=0)

def marginal_repeat_uniform_transform(x,window_size):
    margin_size = int(window_size/2)
    x = gen_repeat_matrix(x,margin_size)
    # h,w = x.shape
    ret = numpy.zeros_like(x)
    scipy.ndimage.filters.uniform_filter(x,window_size,ret)
    # print(ret)
    return ret[margin_size:-margin_size,margin_size:-margin_size]

def marginal_discard_uniform_transform(x,window_size):
    ret = numpy.zeros_like(x)
    filters.uniform_filter(x, window_size, ret)
    return ret
    


In [18]:
def is_zero_matrix(M):
    return (not numpy.any(M))

class NccCalculator:
    def __init__(self,img1,img2,window_width,max_disparity,vertical_window=False):
        self.h,self.w = img1.shape
        if not img1.shape == img2.shape:
            raise ValueError(f"ncc_calculter: The shape of img1 and img2 must be same, but got {img1.shape} and {img2.shape}.")
        
        self.window_width = window_width
        self.margin_width = int(window_width/2)
        self.axis_width = window_width - 2* self.margin_width # 0 or 1

        self.img1 = gen_padded_matrix(img1,self.margin_width,vertical_window)
        self.img2 = gen_padded_matrix(img2,self.margin_width,vertical_window)
        self.img1 = numpy.concatenate([
            self.img1,
            numpy.zeros((self.img1.shape[0],max_disparity),dtype=self.img1.dtype)]
            ,axis=1)
        
        self.mean1 = marginal_discard_uniform_transform(self.img1,window_width)
        self.mean2 = marginal_discard_uniform_transform(self.img2,window_width)
        self.diff1 = self.img1-self.mean1
        self.diff2 = self.img2-self.mean2
        
    def cal_ncc_for_pixel(self,x,y,d):
        window1 =(
            slice(x,x+2*self.margin_width+self.axis_width,None),
            slice(y+d,y+2*self.margin_width+self.axis_width+d,None))
        window2 =(
            slice(x,x+2*self.margin_width+self.axis_width,None),
            slice(y,y+2*self.margin_width+self.axis_width,None))
        numerator = numpy.sum(self.diff1[window1]*self.diff2[window2])
        denominator1 = numpy.sum(numpy.square(self.diff1[window1]))
        denominator2 = numpy.sum(numpy.square(self.diff2[window2]))
        similarity = numerator / math.sqrt(denominator1*denominator2)
        return similarity

    def cal_ncc_for_pixel_2(self,x,y,d):
        window1 =(
            slice(x,x+2*self.margin_width+self.axis_width,None),
            slice(y+d,y+2*self.margin_width+self.axis_width+d,None))
        window2 =(
            slice(x,x+2*self.margin_width+self.axis_width,None),
            slice(y,y+2*self.margin_width+self.axis_width,None))
        diff1 = self.img1[window1]-self.mean1[x,y+d]
        diff2 = self.img2[window2]-self.mean2[x,y]
        numerator = numpy.sum(diff1*diff2)
        if is_zero_matrix(diff1) or is_zero_matrix(diff2):
            return 0
        denominator1 = numpy.sum(numpy.square(diff1))
        denominator2 = numpy.sum(numpy.square(diff2))
        similarity = numerator / math.sqrt(denominator1*denominator2)
        return similarity

    def cal_ncc_for_pixel_v(self,x,y,d):
        window1 =(
            slice(x,x+4*self.margin_width+self.axis_width,None),
            slice(y+d,y+2*self.margin_width+self.axis_width+d,None))
        window2 =(
            slice(x,x+4*self.margin_width+self.axis_width,None),
            slice(y,y+2*self.margin_width+self.axis_width,None))
        diff1 = self.img1[window1]-self.mean1[x,y+d]
        diff2 = self.img2[window2]-self.mean2[x,y]
        numerator = numpy.sum(diff1*diff2)
        if is_zero_matrix(diff1) or is_zero_matrix(diff2):
            return 0
        denominator1 = numpy.sum(numpy.square(diff1))
        denominator2 = numpy.sum(numpy.square(diff2))
        similarity = numerator / math.sqrt(denominator1*denominator2)
        return similarity
        
        

In [23]:
@deprecated(reason="Bad result")
def cal_dsi_by_ncc(img1,img2,window_width,min_disparity,max_disparity):
    '''This function use NCC to calculate DSI
    '''
    h,w = img1.shape
    if not img1.shape == img2.shape:
        raise ValueError(f"cal_dsi_by_ncc: the shape of 2 images should be same, but got {img1.shape} and {img2.shape} instead.")

    # mean value of window for (x,y)
    mean1 = marginal_discard_uniform_transform(img1,window_width)
    mean2 = marginal_discard_uniform_transform(img2,window_width)

    # difference of the pixel to the mean value of its window
    diff1 = img1-mean1
    diff2 = img2-mean2
    # numerator_map = diff1 * diff2
    denominator_map1 = marginal_repeat_uniform_transform(diff1 * diff1,window_width)
    denominator_map2 = marginal_repeat_uniform_transform(diff2 * diff2,window_width)

    dsi = numpy.zeros((h,w,max_disparity-min_disparity))
    for d in range(min_disparity,max_disparity): 
        # move the left image to right, calculate the matchness
        numerator = marginal_repeat_uniform_transform(numpy.roll(diff1,d,axis=1)*diff2,window_width)
        denominator_1 = numpy.roll(denominator_map1,d,axis=1)
        # denominator_1 = marginal_repeat_uniform_transform(numpy.roll(diff1,d,axis=1)*numpy.roll(diff1,d,axis=1),window_width)
        dsi[:, :, d-min_disparity] = numerator / numpy.sqrt(denominator_map2 * denominator_1)
    return -dsi

def cal_dsi_by_opencv_ncc(img1,img2,window_width,min_disparity,max_disparity):
    h,w = img1.shape
    if not img1.shape == img2.shape:
        raise ValueError(f"cal_dsi_by_opencv_ncc: the shape of 2 images should be same, but got {img1.shape} and {img2.shape} instead.")
    margin_width = int(window_width/2)
    axis_width = window_width-2*margin_width # either 0 or 1
    img1_extended = gen_padded_matrix(img1,margin_width)
    img2_extended = gen_padded_matrix(img2,margin_width)
    img1_extended = numpy.concatenate([img1_extended,numpy.zeros((img1_extended.shape[0],max_disparity),dtype=img1_extended.dtype)],axis=1)
    
    dsi = numpy.zeros((h,w,max_disparity-min_disparity),dtype=numpy.float16)
    for x in range(h):
        for y in range(w):
            for d in range(min_disparity,max_disparity):
                window1 = img1_extended[x:x+2*margin_width+axis_width,y+d:y+2*margin_width+axis_width+d]
                window2 = img2_extended[x:x+2*margin_width+axis_width,y:y+2*margin_width+axis_width]
                dsi[x,y,d-min_disparity] = cv2.matchTemplate(window2, window1, cv2.TM_CCOEFF_NORMED)

    return -dsi

def cal_dsi_by_ncc_2(img1,img2,window_width,min_disparity,max_disparity):
    ncc_calculator = NccCalculator(img1,img2,window_width,max_disparity)
    h,w = img1.shape
    dsi = numpy.zeros((h,w,max_disparity-min_disparity),dtype=numpy.float16)
    for x in range(h):
        for y in range(w):
            for d in range(min_disparity,max_disparity):
                dsi[x,y,d-min_disparity] = ncc_calculator.cal_ncc_for_pixel_2(x,y,d)

    return -dsi

@deprecated(reason="no improvement")
def cal_dsi_by_ncc_v(img1,img2,window_width,min_disparity,max_disparity):
    ncc_calculator = NccCalculator(img1,img2,window_width,max_disparity)
    h,w = img1.shape
    dsi = numpy.zeros((h,w,max_disparity-min_disparity),dtype=numpy.float16)
    for x in range(h):
        for y in range(w):
            for d in range(min_disparity,max_disparity):
                dsi[x,y,d-min_disparity] = ncc_calculator.cal_ncc_for_pixel_v(x,y,d)

    return -dsi

In [ ]:
configs = [[90,6,15],[15,6,50],[15,6,15],[15,6,7],[15,6,11],[15,4,11],[15,10,11]]
configs_v = [[15,4,11]]
def save_img(img,path):
    f_path = "/".join(os.path.split(path)[:-1])
    if not os.path.exists(f_path):
        os.makedirs(f_path)
    imageio.imsave(path,img)
    print(path)
    pylab.imshow(img)
    pylab.show()
    pylab.clf()

for img_i,path_pair in enumerate(path):
    im_l = numpy.ma.array(Image.open(path_pair[0]).convert('L'), 'f')
    im_r = numpy.ma.array(Image.open(path_pair[1]).convert('L'), 'f')
    # print(im_l.shape)
    pylab.imshow(im_l)
    pylab.show()
    pylab.clf()
    for config_i,config in enumerate(configs):
        
        max_bias = config[0] # max bias / search length
        start = config[1]
        wid = config[2]

        dmap = cal_dsi_by_opencv_ncc(im_l, im_r, wid, start, start+max_bias)
        res = numpy.argmin(dmap, axis=2)
        save_img(res,f'/gdrive/MyDrive/stereo_vision/results/fig{img_i}config{config_i}_depth1.png')

        dmap = cal_dsi_by_ncc_2(im_l, im_r, wid, start, start+max_bias)
        res = numpy.argmin(dmap, axis=2)
        save_img(res,f'/gdrive/MyDrive/stereo_vision/results/fig{img_i}config{config_i}_depth3.png')

    for config_i,config in enumerate(configs_v):
        max_bias = config[0] # max bias / search length
        start = config[1]
        wid = config[2]

        dmap = cal_dsi_by_ncc_v(im_l, im_r, wid, start, start+max_bias)
        res = numpy.argmin(dmap, axis=2)
        save_img(res,f'/gdrive/MyDrive/stereo_vision/results/v/fig{img_i}config{config_i}_depth3.png')

        